In [1]:
pip install spotipy 

     |████████████████████████████████| 153kB 6.7MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="a006ea8174bc4689b4eb39c47b5449a1" 
secret = "1cca3d1fff6145fdaee72ba822e8b586"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [56]:
import timeit
import pandas as pd
import torch

In [4]:
def getTracks(query, number):
  start = timeit.default_timer()

  # create empty lists where the results are going to be stored
  artist_name = []
  track_name = []
  popularity = []
  track_id = []

  for i in range(0,number,10):
      track_results = sp.search(q=query, type='track', limit=10,offset=i)
      for i, t in enumerate(track_results['tracks']['items']):
          artist_name.append(t['artists'][0]['name'])
          track_name.append(t['name'])
          track_id.append(t['id'])
          popularity.append(t['popularity'])
        

  stop = timeit.default_timer()
  print ('Time to run this code (in seconds):', stop - start)
  df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
  return df_tracks

In [5]:
df_calm_tracks = getTracks('label:calm',200)

Time to run this code (in seconds): 2.4338528389999965


In [6]:
print(df_calm_tracks.shape)
df_calm_tracks.head()

(200, 4)


,artist_name,track_name,track_id,popularity
0,The Calm Factory,Forest Thunderstorm,3NTAcEKTrL3yHnEzBklXcN,63
1,The Calm Factory,Calm Before The Storm,69mYHeNk3DSNIRBUCLCFZI,60
2,The Calm Factory,Rain For Relaxation,5WAqDgplhuoFBG4IUdEP4J,59
3,The Calm Factory,Chuva soa para dormir,5sMzZKg6SLtG5lcudEpDLC,58
4,Sounds Of Calm,Small River On Rocks,1iwAjsOxILo4p6uOZLD0Fa,47


In [7]:
df_energetic_tracks = getTracks('label:energetic',200)

Time to run this code (in seconds): 2.363126795999989


In [8]:
print(df_energetic_tracks.shape)
df_energetic_tracks.head()

(200, 4)


,artist_name,track_name,track_id,popularity
0,Lil Key,Crazy (feat. J.I. & D Low),5XgfOmcWvkXTDmOfO81NRP,14
1,Energetic Alchemy,The Lovers,6cVj2hWi6hDovzI6fjWH3N,9
2,Sharon Hart,Near Reject Pick,3d0t8krXtXnNsCPXruNWqR,9
3,Kasperli,De Seegeischt im Führwehrweiher - Teil 10,1dkMVDHOBz2r171zApCVdr,10
4,Kasperli,S verzauberete Flugzüüg - Teil 3,2i1yUnMB9c1pXjtf7mtRJS,10


In [9]:
df_happy_tracks = getTracks('label:happy',200)

Time to run this code (in seconds): 1.7914993679999895


In [10]:
print(df_happy_tracks.shape)
df_happy_tracks.head()

(200, 4)


,artist_name,track_name,track_id,popularity
0,L.B. One,My Mother Told Me - Vikings Anthem,6pAzrdqZKo4AnRWDKStG7R,65
1,The Happy Fits,Too Late,3LbO439QNr3IX7Ho39Fq8X,59
2,Lil Happy Lil Sad,Let Me Die,46T7puWlH8ptC6lx7bQ3TY,66
3,The Happy Fits,Dirty Imbecile,5erGUceK8ieHWtTXdnGP65,57
4,Sad Frosty,Beavis & Butt-head,0Y9kRbsQKC9zSmfzxPgYiT,56


In [11]:
df_sad_tracks = getTracks('label:sad',200)

Time to run this code (in seconds): 1.6033500069999889


In [12]:
print(df_sad_tracks.shape)
df_sad_tracks.head()

(200, 4)


,artist_name,track_name,track_id,popularity
0,Lil Happy Lil Sad,Let Me Die,46T7puWlH8ptC6lx7bQ3TY,66
1,Sad Monologue,Cabin Jam,3McFNMvwAOUOCM83wyTpUr,59
2,The Licks,Lavender Kiss,70oY6gKgHfMlVLVxFROeiA,57
3,Ramiro sad,Abismo,0WeBxkwIGnnoPAPNsnfEAd,50
4,Sad Ul,Wasteland Worry,0YsE50FuGtzivL2t8CeG9h,44


In [13]:
def cleanDuplicates(tracks):
  grouped = tracks.groupby(['artist_name','track_name'], as_index=True).size()
  grouped[grouped > 1].count()
  tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)
  

In [14]:
cleanDuplicates(df_calm_tracks)

In [15]:
grouped_after_dropping = df_calm_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

In [16]:
cleanDuplicates(df_energetic_tracks)
cleanDuplicates(df_happy_tracks)
cleanDuplicates(df_sad_tracks)

In [17]:
grouped_after_dropping = df_energetic_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

In [18]:
grouped_after_dropping = df_sad_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

In [19]:
def getAudioFeatures(tracks):
  # again measuring the time
  start = timeit.default_timer()

  # empty list, batchsize and the counter for None results
  rows = []
  batchsize = 100
  None_counter = 0

  for i in range(0,len(tracks['track_id']),batchsize):
      batch = tracks['track_id'][i:i+batchsize]
      feature_results = sp.audio_features(batch)
      for i, t in enumerate(feature_results):
          if t == None:
              None_counter = None_counter + 1
          else:
              rows.append(t)
              
  print('Number of tracks where no audio features were available:',None_counter)

  stop = timeit.default_timer()
  print ('Time to run this code (in seconds):',stop - start)
  df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
  return df_audio_features

In [20]:
df_calm_audio_features = getAudioFeatures(df_calm_tracks)

Number of tracks where no audio features were available: 8
Time to run this code (in seconds): 2.3287680340000065


In [21]:
print("Shape of the dataset:", df_calm_audio_features.shape)
df_calm_audio_features.head()

Shape of the dataset: (190, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.142,0.516,1,-25.856,1,0.0694,0.730000,0.944,0.223,0.0263,88.277,audio_features,3NTAcEKTrL3yHnEzBklXcN,spotify:track:3NTAcEKTrL3yHnEzBklXcN,https://api.spotify.com/v1/tracks/3NTAcEKTrL3y...,https://api.spotify.com/v1/audio-analysis/3NTA...,179972,4
1,0.142,0.320,1,-32.837,1,0.0691,0.782000,0.963,0.103,0.0373,139.713,audio_features,69mYHeNk3DSNIRBUCLCFZI,spotify:track:69mYHeNk3DSNIRBUCLCFZI,https://api.spotify.com/v1/tracks/69mYHeNk3DSN...,https://api.spotify.com/v1/audio-analysis/69mY...,241556,4
2,0.194,0.952,7,-17.896,1,0.0888,0.002150,0.996,0.687,0.0309,153.467,audio_features,5WAqDgplhuoFBG4IUdEP4J,spotify:track:5WAqDgplhuoFBG4IUdEP4J,https://api.spotify.com/v1/tracks/5WAqDgplhuoF...,https://api.spotify.com/v1/audio-analysis/5WAq...,157304,3
3,0.248,0.963,9,-18.139,1,0.0865,0.000776,0.987,0.365,0.0289,76.466,audio_features,5sMzZKg6SLtG5lcudEpDLC,spotify:track:5sMzZKg6SLtG5lcudEpDLC,https://api.spotify.com/v1/tracks/5sMzZKg6SLtG...,https://api.spotify.com/v1/audio-analysis/5sMz...,140142,3
4,0.288,0.949,0,-27.451,1,0.1100,0.211000,0.929,0.882,0.0227,89.076,audio_features,1iwAjsOxILo4p6uOZLD0Fa,spotify:track:1iwAjsOxILo4p6uOZLD0Fa,https://api.spotify.com/v1/tracks/1iwAjsOxILo4...,https://api.spotify.com/v1/audio-analysis/1iwA...,243533,4


In [22]:
df_energetic_audio_features = getAudioFeatures(df_energetic_tracks)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 0.492819360999988


In [23]:
print("Shape of the dataset:", df_energetic_audio_features.shape)
df_energetic_audio_features.head()

Shape of the dataset: (200, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.790,0.703,7,-4.285,1,0.1150,0.20700,0.000000,0.1140,0.399,139.995,audio_features,5XgfOmcWvkXTDmOfO81NRP,spotify:track:5XgfOmcWvkXTDmOfO81NRP,https://api.spotify.com/v1/tracks/5XgfOmcWvkXT...,https://api.spotify.com/v1/audio-analysis/5Xgf...,292781,4
1,0.466,0.680,11,-6.729,0,0.0408,0.01540,0.000047,0.1500,0.221,99.943,audio_features,6cVj2hWi6hDovzI6fjWH3N,spotify:track:6cVj2hWi6hDovzI6fjWH3N,https://api.spotify.com/v1/tracks/6cVj2hWi6hDo...,https://api.spotify.com/v1/audio-analysis/6cVj...,216059,4
2,0.811,0.901,11,-5.390,0,0.0389,0.00211,0.848000,0.0425,0.725,127.994,audio_features,3d0t8krXtXnNsCPXruNWqR,spotify:track:3d0t8krXtXnNsCPXruNWqR,https://api.spotify.com/v1/tracks/3d0t8krXtXnN...,https://api.spotify.com/v1/audio-analysis/3d0t...,131250,4
3,0.737,0.434,6,-14.013,1,0.9020,0.54000,0.000000,0.3320,0.634,101.835,audio_features,1dkMVDHOBz2r171zApCVdr,spotify:track:1dkMVDHOBz2r171zApCVdr,https://api.spotify.com/v1/tracks/1dkMVDHOBz2r...,https://api.spotify.com/v1/audio-analysis/1dkM...,115693,4
4,0.718,0.365,1,-12.127,0,0.8900,0.74900,0.000000,0.0926,0.684,83.722,audio_features,2i1yUnMB9c1pXjtf7mtRJS,spotify:track:2i1yUnMB9c1pXjtf7mtRJS,https://api.spotify.com/v1/tracks/2i1yUnMB9c1p...,https://api.spotify.com/v1/audio-analysis/2i1y...,108387,4


In [24]:
df_happy_audio_features = getAudioFeatures(df_happy_tracks)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 3.735616331000017


In [25]:
print("Shape of the dataset:", df_happy_audio_features.shape)
df_happy_audio_features.head()

Shape of the dataset: (200, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.650,0.971,5,-4.088,0,0.2090,0.00656,0.0327,0.2580,0.200,139.970,audio_features,6pAzrdqZKo4AnRWDKStG7R,spotify:track:6pAzrdqZKo4AnRWDKStG7R,https://api.spotify.com/v1/tracks/6pAzrdqZKo4A...,https://api.spotify.com/v1/audio-analysis/6pAz...,197250,4
1,0.685,0.554,7,-10.473,1,0.1150,0.44300,0.0000,0.4820,0.789,160.002,audio_features,3LbO439QNr3IX7Ho39Fq8X,spotify:track:3LbO439QNr3IX7Ho39Fq8X,https://api.spotify.com/v1/tracks/3LbO439QNr3I...,https://api.spotify.com/v1/audio-analysis/3LbO...,201258,4
2,0.503,0.370,11,-17.502,1,0.0489,0.76800,0.5990,0.1160,0.102,78.963,audio_features,46T7puWlH8ptC6lx7bQ3TY,spotify:track:46T7puWlH8ptC6lx7bQ3TY,https://api.spotify.com/v1/tracks/46T7puWlH8pt...,https://api.spotify.com/v1/audio-analysis/46T7...,218776,4
3,0.834,0.531,3,-10.450,1,0.1600,0.48000,0.0000,0.0957,0.390,99.999,audio_features,5erGUceK8ieHWtTXdnGP65,spotify:track:5erGUceK8ieHWtTXdnGP65,https://api.spotify.com/v1/tracks/5erGUceK8ieH...,https://api.spotify.com/v1/audio-analysis/5erG...,239612,4
4,0.848,0.475,0,-8.949,1,0.4760,0.01380,0.0000,0.1070,0.195,138.917,audio_features,0Y9kRbsQKC9zSmfzxPgYiT,spotify:track:0Y9kRbsQKC9zSmfzxPgYiT,https://api.spotify.com/v1/tracks/0Y9kRbsQKC9z...,https://api.spotify.com/v1/audio-analysis/0Y9k...,124317,4


In [26]:
df_sad_audio_features = getAudioFeatures(df_sad_tracks)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 0.48390145700000176


In [27]:
print("Shape of the dataset:", df_sad_audio_features.shape)
df_sad_audio_features.head()

Shape of the dataset: (198, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.503,0.370,11,-17.502,1,0.0489,0.76800,0.5990,0.1160,0.102,78.963,audio_features,46T7puWlH8ptC6lx7bQ3TY,spotify:track:46T7puWlH8ptC6lx7bQ3TY,https://api.spotify.com/v1/tracks/46T7puWlH8pt...,https://api.spotify.com/v1/audio-analysis/46T7...,218776,4
1,0.477,0.370,4,-12.394,1,0.0315,0.68900,0.7990,0.0977,0.638,179.707,audio_features,3McFNMvwAOUOCM83wyTpUr,spotify:track:3McFNMvwAOUOCM83wyTpUr,https://api.spotify.com/v1/tracks/3McFNMvwAOUO...,https://api.spotify.com/v1/audio-analysis/3McF...,192710,4
2,0.578,0.115,0,-20.333,1,0.0360,0.00922,0.0031,0.0834,0.363,199.975,audio_features,70oY6gKgHfMlVLVxFROeiA,spotify:track:70oY6gKgHfMlVLVxFROeiA,https://api.spotify.com/v1/tracks/70oY6gKgHfMl...,https://api.spotify.com/v1/audio-analysis/70oY...,243646,3
3,0.677,0.416,0,-11.775,1,0.0313,0.42100,0.0000,0.1350,0.172,126.174,audio_features,0WeBxkwIGnnoPAPNsnfEAd,spotify:track:0WeBxkwIGnnoPAPNsnfEAd,https://api.spotify.com/v1/tracks/0WeBxkwIGnno...,https://api.spotify.com/v1/audio-analysis/0WeB...,174762,4
4,0.543,0.788,5,-0.949,0,0.3180,0.12900,0.0000,0.4210,0.468,142.784,audio_features,0YsE50FuGtzivL2t8CeG9h,spotify:track:0YsE50FuGtzivL2t8CeG9h,https://api.spotify.com/v1/tracks/0YsE50FuGtzi...,https://api.spotify.com/v1/audio-analysis/0YsE...,64016,4


In [28]:
def dropColumns(df_audio_features):
  columns_to_drop = ['analysis_url','track_href','type']
  df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

  df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

  df_audio_features.shape

In [29]:
dropColumns(df_calm_audio_features)
dropColumns(df_energetic_audio_features)
dropColumns(df_happy_audio_features)
dropColumns(df_sad_audio_features)

In [46]:
if not 'label' in df_calm_audio_features:
  df_calm_audio_features['label'] = 1 # label calm with 1

if not 'label' in df_energetic_audio_features:
  df_energetic_audio_features['label'] = 2 # label energetic with 2

if not 'label' in df_happy_audio_features:
  df_happy_audio_features['label'] = 3 # label happy with 3

if not 'label' in df_sad_audio_features:
  df_sad_audio_features['label'] = 4 # label sad with 4

In [33]:
def mergeDataframes(df_tracks,df_audio_features):
  # merge both dataframes
  # the 'inner' method will make sure that we only keep track IDs present in both datasets
  df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
  
  return df

In [47]:
df_calm = mergeDataframes(df_calm_tracks, df_calm_audio_features)
df_energetic = mergeDataframes(df_energetic_tracks, df_energetic_audio_features)
df_happy = mergeDataframes(df_happy_tracks, df_happy_audio_features)
df_sad = mergeDataframes(df_sad_tracks, df_sad_audio_features)

In [35]:
df_calm.to_csv('calm.csv')
df_energetic.to_csv('energetic.csv')
df_happy.to_csv('happy.csv')
df_sad.to_csv('sad.csv')

In [36]:
%mkdir data_from_spotify

In [37]:
!mv "/content/calm.csv" "/content/data_from_spotify"

In [38]:
!mv "/content/energetic.csv" "/content/data_from_spotify"

In [39]:
!mv "/content/happy.csv" "/content/data_from_spotify"

In [40]:
!mv "/content/sad.csv" "/content/data_from_spotify"

In [49]:
df_list = [df_calm, df_energetic, df_happy, df_sad]

In [50]:
df = pd.concat(df_list)

In [51]:
df

,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,time_signature,label
0,The Calm Factory,Forest Thunderstorm,3NTAcEKTrL3yHnEzBklXcN,63,0.142,0.516,1,-25.856,1,0.0694,0.730000,0.944000,0.2230,0.0263,88.277,spotify:track:3NTAcEKTrL3yHnEzBklXcN,179972,4,1
1,The Calm Factory,Calm Before The Storm,69mYHeNk3DSNIRBUCLCFZI,60,0.142,0.320,1,-32.837,1,0.0691,0.782000,0.963000,0.1030,0.0373,139.713,spotify:track:69mYHeNk3DSNIRBUCLCFZI,241556,4,1
2,The Calm Factory,Rain For Relaxation,5WAqDgplhuoFBG4IUdEP4J,59,0.194,0.952,7,-17.896,1,0.0888,0.002150,0.996000,0.6870,0.0309,153.467,spotify:track:5WAqDgplhuoFBG4IUdEP4J,157304,3,1
3,The Calm Factory,Chuva soa para dormir,5sMzZKg6SLtG5lcudEpDLC,58,0.248,0.963,9,-18.139,1,0.0865,0.000776,0.987000,0.3650,0.0289,76.466,spotify:track:5sMzZKg6SLtG5lcudEpDLC,140142,3,1
4,Sounds Of Calm,Small River On Rocks,1iwAjsOxILo4p6uOZLD0Fa,47,0.288,0.949,0,-27.451,1,0.1100,0.211000,0.929000,0.8820,0.0227,89.076,spotify:track:1iwAjsOxILo4p6uOZLD0Fa,243533,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,J-Soul,Ride for Me,4BFCNjiYvViuDgF8qvWNqZ,28,0.399,0.565,1,-8.901,0,0.4510,0.243000,0.000002,0.1020,0.6000,200.295,spotify:track:4BFCNjiYvViuDgF8qvWNqZ,268774,3,4
194,Sad Puppy,Getting Higher,1QXKnnSbTlNJsehovOCz5X,37,0.663,0.914,5,-6.091,0,0.0642,0.022200,0.000023,0.1020,0.9190,123.977,spotify:track:1QXKnnSbTlNJsehovOCz5X,161855,4,4
195,yesterday,i hope you feel the same in the morning,3VB4hN65b00EOW3KYMQVWV,23,0.713,0.506,1,-9.404,0,0.1100,0.756000,0.000003,0.1050,0.3300,140.182,spotify:track:3VB4hN65b00EOW3KYMQVWV,251751,4,4
196,Lil Happy Lil Sad,Feel Low,3HvLIpY1lnhoj8aO4Pptde,30,0.702,0.163,9,-13.185,0,0.0303,0.860000,0.010600,0.1120,0.1500,103.036,spotify:track:3HvLIpY1lnhoj8aO4Pptde,269709,4,4


In [53]:
df = df.sample(frac=1).reset_index(drop=True)

In [54]:
df

,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,time_signature,label
0,How Sad,Hot Blur,08OTqB3xjsLa5EZb9Fu2Lw,41,0.683,0.8240,1,-8.569,0,0.2220,0.024900,0.627000,0.0533,0.9180,154.008,spotify:track:08OTqB3xjsLa5EZb9Fu2Lw,201049,4,4
1,Baby Wars,"Anakin's Betrayal ""from Star Wars"" - Lullaby V...",4xirTNd21iFPlT3Vce3k5z,29,0.584,0.0985,9,-15.685,0,0.0353,0.993000,0.670000,0.0618,0.0707,84.982,spotify:track:4xirTNd21iFPlT3Vce3k5z,172384,4,3
2,Pepe Lienhard Orchestra,Stardust,3ImhKDH5fOrMreY0TUO2Xv,0,0.379,0.1970,3,-14.437,1,0.0270,0.831000,0.902000,0.1080,0.1190,63.561,spotify:track:3ImhKDH5fOrMreY0TUO2Xv,216333,4,2
3,The Happy Return,Lemonade,4NntR4wm7QBB8jtjNZPstP,33,0.512,0.7480,11,-5.940,1,0.0338,0.026600,0.001560,0.2030,0.2870,139.926,spotify:track:4NntR4wm7QBB8jtjNZPstP,208001,4,3
4,sad alex,new heartbreak,7as2G7f005jH4jNMI4UX8A,40,0.639,0.6750,2,-4.296,1,0.0658,0.007420,0.000294,0.3360,0.6290,155.940,spotify:track:7as2G7f005jH4jNMI4UX8A,194615,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,Calm Palm,Hit the Road,0jQDBBWNmLJk5fdIMg0Xfw,33,0.173,0.2750,1,-28.594,1,0.0609,0.098600,0.930000,0.1130,0.2550,106.176,spotify:track:0jQDBBWNmLJk5fdIMg0Xfw,60970,3,1
784,Funk D,Awesome - Original Mix,3vCGMW1g7ne7dpYF0dojMA,0,0.822,0.8110,4,-4.717,0,0.1090,0.000929,0.649000,0.3100,0.0987,128.001,spotify:track:3vCGMW1g7ne7dpYF0dojMA,319028,4,2
785,The Opposite of Sad,2 Days,3tE8NNNHTkj1X64TdnTufK,43,0.704,0.5820,11,-5.831,1,0.0577,0.665000,0.000000,0.1640,0.4920,137.042,spotify:track:3tE8NNNHTkj1X64TdnTufK,170551,4,4
786,Energetic,Stronger Now,4W1OCj5NynJKEHosO9Piru,1,0.439,0.5540,9,-14.935,0,0.0523,0.036500,0.003730,0.1060,0.1290,160.018,spotify:track:4W1OCj5NynJKEHosO9Piru,258000,4,2


In [60]:
# creating the training and test sets

n_all = len(df)
n_train = int(0.8 * n_all)
n_test = n_all - n_train
rng = torch.Generator().manual_seed(1549)
dataset_train, dataset_test = torch.utils.data.random_split(df, [n_train, n_test], rng)

In [61]:
len(dataset_test)

158

In [62]:
len(dataset_train)

630